# Psycopg

Psycopg is a postgres adapter for python programming language.

In [6]:
import time
import docker
import psycopg
from src.rerun_docker import reload_docker_container

client = docker.from_env()
container_name = "psycopg_examples"
container_port = 5432

container = reload_docker_container(
    container_name,
    image="postgres:17.4",
    environment={
        "POSTGRES_PASSWORD": "password",
    },
    ports={f"{container_port}/tcp": 5432},
    detach=True,
    remove=True
)

def get_connection():
    return psycopg.connect(
        dbname="postgres",
        user="postgres",
        password="password",
        host="localhost",
        port=container_port
    )

## Load data

To run anything in postgres you need to:

- Create a connection: `psycopg.connect()`.
- Create cursor: `connection.cursor()`.
- Execute your query: `cursor.execute()`.
- Get data from cursor: `cursor.fetchall`, `cursor.fetachone`, `cursor.description` etc.

---

The following cell connects to the database.

In [2]:
connection = psycopg.connect(
    dbname="postgres",
    user="postgres",
    password="password",
    host="localhost",
    port=container_port
)

The next code executes the query in the database and loads the results and the name of the column.

In [3]:
with connection.cursor() as cursor:
    cursor.execute("SELECT 50 AS value1, 'test' AS value2;")
    columns = cursor.description
    data = cursor.fetchall()

print("Columns", columns)
print("Data", data)

Columns [<Column 'value1', type: int4 (oid: 23)>, <Column 'value2', type: text (oid: 25)>]
Data [(50, 'test')]


### Columns

Information about columns is stored in the `cursor.description` attribute, which is a `list` of objects that describe columns.

---

The next cell shows the type of the object that describes column.

In [11]:
type(columns[0])

psycopg.Column

You can get the name of the column from the `name` attribute.

In [12]:
columns[0].name

'value1'

## No data

Some queries should not to return any data. You can check if the given query returnes any data by checking the `cursor.rowcount` attribute. If it takes `-1` value, it means that query doesn't return anything.

---

The following cell shows the `cursor.rowcount` attribute in the case of a `CREATE TABLE` query.

In [8]:
with connection.cursor() as cursor:
    cursor.execute("CREATE TABLE table1 (val1 INT, val2 INT);")
    rowcount = cursor.rowcount
    cursor.execute("DROP TABLE table1;")

rowcount

-1

As a result, `cursor.rowcount` takes the value `-1`. The next cell shows the same case but for the `SELECT` query to show difference.

In [7]:
with connection.cursor() as cursor:
    cursor.execute("SELECT 50;")
    rowcount = cursor.rowcount

rowcount

1

## Server messages

In some cases postgreSQL server sends some textual information corresponding to the command that caused these responses.

- You can access the messages describing which commands were executed by using `cursor.statusmessage` attribute.
- You can access postgres log messages from special handler that can be added using `connection.add_notice_handler` method.

Check details in the [corresponding section of the official documentation](https://www.psycopg.org/psycopg3/docs/advanced/async.html#server-messages).

---

The following cell shows how you can extract various messagee that appear during the execution of the `DROP TABLE IF EXISTS` command.

In [ ]:
connection = get_connection()

global global_notice

def log_notice(diag):
    print(f"The server says: {diag.severity} - {diag.message_primary}")

connection.add_notice_handler(log_notice)

with connection.cursor() as cursor:
    cursor.execute("DROP TABLE IF EXISTS default_values_example;")
    print(cursor.statusmessage)

connection.close()

The server says: NOTICE - table "default_values_example" does not exist, skipping
DROP TABLE
